## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

2023-04-16 10:28:21.925823: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Inspect the dataframe
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
# One-hot encode the categorical variables
dummy_df = pd.get_dummies(application_df[['APPLICATION_TYPE', 'CLASSIFICATION']])
merged_df = application_df.merge(dummy_df, left_index=True, right_index=True)
merged_df = merged_df.drop(['APPLICATION_TYPE', 'CLASSIFICATION'], axis=1)

In [11]:
# Split our preprocessed data into our features and target arrays
X = merged_df.drop('IS_SUCCESSFUL', axis=1).values
y = merged_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [12]:
import numpy as np

# Convert X_train and X_test back to a DataFrame
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

# Check data types of columns in X_train
column_data_types = X_train.dtypes

# Identify categorical columns
cat_cols = column_data_types[column_data_types == 'object'].index.tolist()

# Print the list of categorical columns
print("Categorical columns:", cat_cols)

Categorical columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


In [13]:
# Check the shape of X_train and X_test
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (25724, 23)
X_test shape: (8575, 23)


In [14]:
# Convert categorical data to numeric with pd.get_dummies
# One-hot encode the categorical variables in X_train and X_test
for col in cat_cols:
    if col in X_train.columns:
        X_train = pd.get_dummies(X_train, columns=[col], prefix=[col], drop_first=True)
    if col in X_test.columns:
        X_test = pd.get_dummies(X_test, columns=[col], prefix=[col], drop_first=True)
        
# Align the columns in X_train and X_test
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/var/folders/sq/w5nrx3592fl25n_hv7_dbx8w0000gn/T/ipykernel_47659/1674258162.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X_train = pd.get_dummies(X_train, columns=[col], prefix=[col], drop_first=True)
/var/folders/sq/w5nrx3592fl25n_hv7_dbx8w0000gn/T/ipykernel_47659/1674258162.py:7: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X_test = pd.get_dummies(X_test, columns=[col], prefix=[col], drop_first=True)
/var/folders/sq/w5nrx3592fl25n_hv7_dbx8w0000gn/T/ipykernel_47659/1674258162.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X_train = pd.get_dummies(X_train, columns=[col], prefix=[col], drop_first=True)
/var/folders/sq/w5nrx3592fl25n_hv7_dbx8w0000gn/T/ipykernel_4

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = Sequential()

# First hidden layer
nn.add(Dense(units=10, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(Dense(units=5, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                66060     
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 66,121
Trainable params: 66,121
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
804/804 [==============================] - 2s 1ms/step - loss: 0.7351 - accuracy: 0.5815
Epoch 2/10
804/804 [==============================] - 1s 1ms/step - loss: 0.5144 - accuracy: 0.7790
Epoch 3/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4389 - accuracy: 0.8028
Epoch 4/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4245 - accuracy: 0.8061
Epoch 5/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4195 - accuracy: 0.8079
Epoch 6/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4152 - accuracy: 0.8084
Epoch 7/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4111 - accuracy: 0.8092
Epoch 8/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4085 - accuracy: 0.8096
Epoch 9/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4057 - accuracy: 0.8096
Epoch 10/10
804/804 [==============================] - 1s 1ms/step - loss: 0.4047 - accuracy: 0.8090

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6315 - accuracy: 0.7108 - 326ms/epoch - 1ms/step
Loss: 0.6314917206764221, Accuracy: 0.7107871770858765


In [19]:
# Export our model to HDF5 file
nn.save('model.h5')